## Summary 
   
> About the data analysis
>
> -I worked with the OSHA accident database (osha-accident).
>
> -The analysis was done in pandas / Python. I also used Excel to do some minor calculations.
>
> -The results correspond to a five-year period, from 2017 to 2021.
>
> -I used calculations of percentage of the overall figures and change over time.


> Ideas to be explored: 
>>1.The number of accident reports decreased by 83% from 2017 to 2021. This could be related to changes on how incidents were reported during the Trump Administration. A possible focus of investigation could be to look at companies that were usually investigated by OSHA. The goal would be to show how the Trump administration's policies benefited them.
>
> 
>> The analysis of accident causes showed that fatal accidents caused by falls from significant heights (falls from a roof or ladder) have been the most frequent causes of death in the construction industry, after deaths due to covid.  Why are falls still a major cause of death? What is going wrong in the industry? Are workers receiving sufficient training? Do workers have access to the necessary equipment? Which companies are reporting the most problems? 
   


## Set up

In [138]:
import requests
import pandas as pd

In [114]:
import numpy as np

In [115]:
from datetime import datetime
from datetime import date

In [116]:
from collections import Counter

### Get the data | OSHA accidents

In [117]:
pwd

'/Users/monicacordero/Documents/lede_master/IM2'

### OSHA accidents 

In [118]:
# osha accident df.describe() me da mean, min
df= pd.read_csv("osha_accident.csv")
df.head(1)

,summary_nr,report_id,event_date,event_time,event_desc,event_keyword,const_end_use,build_stories,nonbuild_ht,project_cost,project_type,sic_list,fatality,state_flag,abstract_text,load_dt
0,9407,112600,1984-03-25 00:00:00,NaN,NONE,NaN,NaN,NaN,NaN,NaN,NaN,2621,X,NaN,NaN,2019-07-22 00:18:27 EDT


In [119]:
# number of rows and column types
df.shape

(142981, 16)

In [120]:
## Cheking Columns, Non-Null, type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142981 entries, 0 to 142980
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   summary_nr     142981 non-null  int64  
 1   report_id      142981 non-null  int64  
 2   event_date     142981 non-null  object 
 3   event_time     0 non-null       float64
 4   event_desc     142979 non-null  object 
 5   event_keyword  142713 non-null  object 
 6   const_end_use  30759 non-null   object 
 7   build_stories  21528 non-null   float64
 8   nonbuild_ht    15317 non-null   float64
 9   project_cost   25498 non-null   object 
 10  project_type   31130 non-null   object 
 11  sic_list       109089 non-null  object 
 12  fatality       64554 non-null   object 
 13  state_flag     0 non-null       float64
 14  abstract_text  0 non-null       float64
 15  load_dt        142981 non-null  object 
dtypes: float64(5), int64(2), object(9)
memory usage: 17.5+ MB


In [121]:
## checking and filter by event_date column 
df.sort_values(by = ["event_date"],ascending = False)
df["event_date"] = pd.to_datetime(df["event_date"])
df_filtered = df[df["event_date"]>"2017-1-1"]

In [122]:
df_filtered.head(2)

,summary_nr,report_id,event_date,event_time,event_desc,event_keyword,const_end_use,build_stories,nonbuild_ht,project_cost,project_type,sic_list,fatality,state_flag,abstract_text,load_dt
114381,220915433,111400,2017-03-15,NaN,EMPLOYEE KILLED IN DIVING ACCIDENT,"ASPHYXIATED,DIVER,DROWN,EMERGENCY RESPONSE,HOI...",NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,2022-03-20 00:18:48 EDT
114421,220917504,420600,2017-01-09,NaN,EMPLOYEE FRACTURES FINGER WHEN HIS FINGER IS C...,"CAUGHT BETWEEN,EQUIPMENT OPERATOR,FINGER,FRACT...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-20 00:18:47 EDT


In [123]:
df_filtered.sort_values(by=["event_date"]).head(2)


,summary_nr,report_id,event_date,event_time,event_desc,event_keyword,const_end_use,build_stories,nonbuild_ht,project_cost,project_type,sic_list,fatality,state_flag,abstract_text,load_dt
117821,220966246,552652,2017-01-02,NaN,EMPLOYEE'S LEFT INDEX AND MIDDLE FINGER TIPS A...,"AMPUTATED,CHAIN,CONVEYOR,FINGER,HAND,MACHINE G...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-20 00:18:47 EDT
114646,220923155,317500,2017-01-02,NaN,EMPLOYEE PINCHES FINGER THEN FAINTS STRIKING H...,"CAUGHT BETWEEN,CAUGHT IN,CONCUSSION,FAINTED,FI...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-20 00:18:46 EDT


In [124]:
# extract year and month

In [125]:
df_filtered["event_year"] = pd.to_datetime(df["event_date"]).dt.year
df_filtered["event_month"] = pd.to_datetime(df["event_date"]).dt.month

/Users/monicacordero/.pyenv/versions/3.8.10/lib/python3.8/site-packages/pandas-1.4.0.dev0+143.g5675cd8ab2-py3.8-macosx-11.3-arm64.egg/pandas/core/frame.py:3602: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/Users/monicacordero/.pyenv/versions/3.8.10/lib/python3.8/site-packages/pandas-1.4.0.dev0+143.g5675cd8ab2-py3.8-macosx-11.3-arm64.egg/pandas/core/frame.py:3602: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [126]:
df_filtered.shape

(27497, 18)

In [127]:
df_filtered.dtypes

summary_nr                int64
report_id                 int64
event_date       datetime64[ns]
event_time              float64
event_desc               object
event_keyword            object
const_end_use            object
build_stories           float64
nonbuild_ht             float64
project_cost             object
project_type             object
sic_list                 object
fatality                 object
state_flag              float64
abstract_text           float64
load_dt                  object
event_year                int64
event_month               int64
dtype: object

### Interrogating the data 

#### Most common events | 2017 - 2021

In [139]:
df_filtered["event_desc"].value_counts().head(20)

EMPLOYEE WITH COVID-19 INFECTION DIES                    1232
EMPLOYEE WITH COVID-19 INFECTION IS HOSPITALIZED          146
EMPLOYEE FALLS FROM ROOF AND IS KILLED                     91
EMPLOYEE IS KILLED IN FALL FROM ROOF                       83
EMPLOYEE FALLS FROM LADDER AND IS KILLED                   61
EMPLOYEE DIES FROM HEART ATTACK                            46
EMPLOYEE IS KILLED IN FALL FROM LADDER                     46
EMPLOYEE FALLS FROM SCAFFOLD AND IS KILLED                 32
EMPLOYEE FALLS THROUGH SKYLIGHT AND IS KILLED              31
EMPLOYEE IS STRUCK AND KILLED BY MOTOR VEHICLE             27
EMPLOYEE IS INJURED IN FALL FROM LADDER                    24
EMPLOYEE SUFFERS HEART ATTACK AND DIES                     24
EMPLOYEE FALLS FROM LADDER AND FRACTURES LEG               24
EMPLOYEE FALLS FROM LADDER AND IS INJURED                  23
EMPLOYEE IS KILLED IN TRENCH COLLAPSE                      22
EMPLOYEE IS INJURED IN FALL FROM ROOF                      21
EMPLOYEE

#### Event per year

In [129]:
# df_filtered["event_desc"].value_counts().head(10)
df_filtered.groupby("event_year")["event_desc"].count()

event_year
2017    8835
2018    9086
2019    4967
2020    3143
2021    1465
2022       1
Name: event_desc, dtype: int64

In [130]:
## fact checking using summary_nr vs event_desc 
df_filtered.groupby("event_year")["summary_nr"].count()

event_year
2017    8835
2018    9086
2019    4967
2020    3143
2021    1465
2022       1
Name: summary_nr, dtype: int64

#### Change over time

In [131]:
df_filtered.groupby("event_year")["summary_nr"].count().pct_change().mul(100) 

event_year
2017          NaN
2018     2.840973
2019   -45.333480
2020   -36.722368
2021   -53.388482
2022   -99.931741
Name: summary_nr, dtype: float64

In [132]:
df3 = df_filtered["event_desc"]
df3.head(2)

114381                   EMPLOYEE KILLED IN DIVING ACCIDENT
114421    EMPLOYEE FRACTURES FINGER WHEN HIS FINGER IS C...
Name: event_desc, dtype: object

In [134]:
## accident_injury
df2 = pd.read_csv("osha_accident_injury.csv")
df2.head(1)

,summary_nr,rel_insp_nr,age,sex,nature_of_inj,part_of_body,src_of_injury,event_type,evn_factor,hum_factor,...,degree_of_inj,task_assigned,hazsub,const_op,const_op_cause,fat_cause,fall_distance,fall_ht,injury_line_nr,load_dt
0,18,10006732,0,NaN,10.0,12.0,15.0,13.0,18.0,1.0,...,1.0,1.0,NaN,0.0,0.0,0.0,NaN,NaN,1,2017-03-20 01:00:11 EDT


In [100]:

df2.shape

(207618, 21)

In [101]:
df2["nature_of_inj"].isna().sum()

35397

In [102]:
df2["nature_of_inj"].sum()

1759457.0

In [136]:
df2["nature_of_inj"].value_counts().head(20)

21.0    36600
12.0    36220
1.0     15662
5.0     12546
2.0     12016
10.0    11475
3.0     10862
6.0      9798
7.0      7371
0.0      5436
17.0     4379
4.0      3137
18.0     1864
20.0     1795
9.0       891
15.0      877
11.0      419
22.0      233
14.0      188
8.0       143
Name: nature_of_inj, dtype: int64

In [135]:
df2["nature_of_inj"].value_counts().head(20)

21.0    36600
12.0    36220
1.0     15662
5.0     12546
2.0     12016
10.0    11475
3.0     10862
6.0      9798
7.0      7371
0.0      5436
17.0     4379
4.0      3137
18.0     1864
20.0     1795
9.0       891
15.0      877
11.0      419
22.0      233
14.0      188
8.0       143
Name: nature_of_inj, dtype: int64